In [1]:
import pickle as pkl
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ref_map, test_map = 986625, 1038115
ref_file = f"sc_node_attributes_{ref_map}.csv"
test_file = f"sc_node_attributes_{test_map}.csv"
def get_node_df(file_name):
    df = pd.read_csv(Path.cwd() / file_name)
    df.sort_values(by=['CD_MemberList_Size'], ascending=False, inplace=True)
    df.index = np.arange(len(df))
    return df
test_node_df = get_node_df(test_file)
ref_node_df = get_node_df(ref_file)

In [3]:
test_node_df.head()

,CD_AnnotatedMembers,CD_AnnotatedMembers_Overlap,CD_AnnotatedMembers_Pvalue,CD_AnnotatedMembers_Size,CD_CommunityName,CD_Labeled,CD_MemberList,CD_MemberList_LogSize,CD_MemberList_Size,CORUM_FDRs,...,GO_CC_terms,HiDeF_persistence,HPA_FDRs,HPA_jaccard_indexes,HPA_overlap_genes,HPA_terms,name,represents,selected,shared name
0,NaN,0.0,0.0,0,NaN,False,NPRL3 MIOS COPS9 RPS14 TOM1L2 AAGAB CUL1 RSL1D...,12.279,4971,NaN,...,GO:0005575,97,NaN,NaN,NaN,NaN,C4971,C4971,False,C4971
1,NaN,0.0,0.0,0,NaN,False,NPRL3 MSTO1 CDC123 ASB1 TOM1L2 RPS14 XPNPEP3 S...,11.573,3047,NaN,...,GO:0005737|GO:0016020|GO:0071944|GO:0012505|GO...,16,5.0713979440993345e-06|1.698752470947653e-22|1...,0.34869138495092694|0.1796652983896432|0.17611...,"WDR7,HEATR5B,NMT1,RAF1,CDK3,UBXN6,PXDNL,TRMT1,...",Cytosol|Vesicles|Plasma membrane|Mitochondria|...,C4972,C4972,False,C4972
2,NaN,0.0,0.0,0,NaN,False,POLR2C MIOS COPS9 RPS14 TRPS1 CUL1 TNPO2 WAPL ...,10.709,1674,NaN,...,GO:0031981|GO:0005654|GO:0005634|GO:0070013|GO...,30,1.114198583138097e-208|4.913898991026879e-27,0.47879011434894875|0.13556531284302964,"KDM1A,FKBP5,RCC1,GMNN,FOXK2,TRMT1,PROX1,DNAJB1...",Nucleoplasm|Nucleoli,C4973,C4973,False,C4973
3,NaN,0.0,0.0,0,NaN,False,MICAL3 SRSF6 POLR2C KPNA6 RPP30 MIOS SKAP1 COP...,9.926,973,6.238836119749178e-07,...,GO:0005654|GO:0031981|GO:0005634|GO:0140513|GO...,20,2.0466748207799003e-174,0.3393939393939394,"KDM1A,FKBP5,RCC1,HNRNPA2B1,PIN1,ZNHIT1,GMEB1,S...",Nucleoplasm,C4974,C4974,False,C4974
4,NaN,0.0,0.0,0,NaN,False,CCDC25 TSPAN15 ABCA12 ASPSCR1 TSN ANXA9 TUSC2 ...,9.640,798,2.136142610069883e-13|9.577330399236416e-11,...,GO:0005829|GO:0005737|GO:0031982|GO:1903561|GO...,13,6.814621552869841e-53,0.23153153153153153,"WDR7,RRAS2,RBMS1,CRYZ,NMT1,SBDS,CDK3,PSMC6,SH3...",Cytosol,C4975,C4975,False,C4975


In [4]:
ref_node_df.head()

,CD_AnnotatedMembers,CD_AnnotatedMembers_Overlap,CD_AnnotatedMembers_Pvalue,CD_AnnotatedMembers_Size,CD_CommunityName,CD_Labeled,CD_MemberList,CD_MemberList_LogSize,CD_MemberList_Size,CORUM_FDRs,...,GO_CC_terms,HiDeF_persistence,HPA_FDRs,HPA_jaccard_indexes,HPA_overlap_genes,HPA_terms,name,represents,selected,shared name
0,NaN,0.0,0.0,0,NaN,False,PDCD5 CSNK1D GPC4 WDR83 SMTN PYCARD IL27RA BOR...,12.279,4971,NaN,...,GO:0005575,98,NaN,NaN,NaN,NaN,C4971,C4971,False,C4971
1,NaN,0.0,0.0,0,NaN,False,CHCHD2 CSGALNACT1 PDCD5 FAM114A2 NAP1L4 NLRP10...,11.777,3509,NaN,...,GO:0005737|GO:0016020|GO:0005829|GO:0012505|GO...,11,2.4644243256692826e-32|5.642147520260271e-29|8...,0.40556711758584807|0.18018018018018017|0.1776...,"WDR7,HEATR5B,NMT1,RAF1,CDK3,UBXN6,PXDNL,TRMT1,...",Cytosol|Vesicles|Plasma membrane|Mitochondria|...,C4972,C4972,False,C4972
2,NaN,0.0,0.0,0,NaN,False,CSGALNACT1 FAM114A2 NAP1L4 PITPNM3 APBB1 TOR1A...,10.876,1879,NaN,...,GO:0016020|GO:0005737|GO:0012505|GO:0031090|GO...,24,2.803385316389283e-13|3.1848585055775276e-22|3...,0.16886064457557876|0.1460564751703992|0.13496...,"HEATR5B,VTI1A,SORCS2,PIK3R3,IQSEC2,POGLUT3,P3H...",Vesicles|Mitochondria|Golgi apparatus,C4973,C4973,False,C4973
3,NaN,0.0,0.0,0,NaN,False,H1-4 RHOBTB1 METTL18 KDM5A POGK SMARCC2 BTBD9 ...,10.674,1634,NaN,...,GO:0031981|GO:0005654|GO:0005634|GO:0070013|GO...,33,2.163633559297123e-198|5.306420742254344e-30,0.466913123844732|0.1399662731871838,"KDM1A,FKBP5,RCC1,GMNN,FOXK2,NPAS1,PROX1,DNAJB1...",Nucleoplasm|Nucleoli,C4974,C4974,False,C4974
4,NaN,0.0,0.0,0,NaN,False,ZNF521 POLE H1-4 CWF19L2 CUL1 RPAP2 ZNF423 ARI...,10.006,1028,1.1533836580586058e-07,...,GO:0005654|GO:0031981|GO:0005634|GO:0070013|GO...,10,5.061988940463486e-179,0.3538152610441767,"KDM1A,FKBP5,RCC1,PIN1,ZNHIT1,GMEB1,SH3BP5,PSMA...",Nucleoplasm,C4975,C4975,False,C4975


In [5]:
# assumes the df has been sorted already so that the whole cell node is on top
sc_genes = test_node_df.iloc[0]["CD_MemberList"].split(" ")
reg_genes = ref_node_df.iloc[0]["CD_MemberList"].split(" ")
print(len(sc_genes), len(reg_genes), len(set(sc_genes) & set(reg_genes)))
print(len(test_node_df), len(ref_node_df))

4971 4971 4971
334 320


In [6]:
from hypergeom_test import run_hypergeo_enrichr

fdr_df = run_hypergeo_enrichr(test_node_df, ref_node_df)

100%|██████████| 334/334 [00:06<00:00, 49.18it/s]


In [15]:
num_enriched = np.sum(fdr_df < 1.0, axis=1)
print(f"{np.sum(num_enriched)}\n")
for i in zip(test_node_df.index):
    enriched_indices = np.where(fdr_df.iloc[i] < 1.0)[0]
    members = set(test_node_df.iloc[i]["CD_MemberList"].split(" "))
    if len(enriched_indices) > 0:
        print(i, enriched_indices)
        print("Node Memberlist:", members)
        # print("Ref Annotions:", ref_df.iloc[enriched_indices]["Annotation"].values)
        print("Ref Annotions:", ref_node_df.iloc[enriched_indices]["GO_CC_descriptions"].values)
        # print("Ref Annotions:", ref_df.iloc[enriched_indices]["corum_terms"].values)
        print("Ref Memberlist:", ref_node_df.iloc[enriched_indices]["CD_MemberList"].values)
        matched_members = []
        for match in ref_node_df.iloc[enriched_indices]["CD_MemberList"].values:
            matched_members.append(members & set(match.split(" ")))
        print("Memberlist Intersection:", matched_members)
        print("FDR:", fdr_df.iloc[i][enriched_indices].values)
        print()

187

0 [0]
Node Memberlist: {'GALK1', 'TNNT1', 'HEBP1', 'HSPB8', 'MRFAP1', 'SERPINA1', 'ATP6V1D', 'SARAF', 'EZH1', 'MCC', 'ZNF114', 'TLN2', 'PIGW', 'CBLB', 'RIPK2', 'MEF2D', 'CAP2', 'WDR5', 'PRIM2', 'SMAD2', 'KLHL21', 'MBD2', 'LAMTOR4', 'MSN', 'SERINC1', 'CBX5', 'RWDD4', 'C12orf43', 'ALG8', 'ATG5', 'ING1', 'RSBN1L', 'TCEAL9', 'EDA', 'SYTL4', 'BEND3', 'SRP54', 'TMLHE', 'NDUFV2', 'YBX3', 'POGLUT3', 'SNAP29', 'CIAPIN1', 'CCNA1', 'PUS7', 'TTI1', 'METTL15', 'UAP1', 'ME2', 'ZBTB21', 'FKBP2', 'MNAT1', 'VPS33A', 'DDX6', 'CORO2A', 'LPL', 'MTFR1', 'EPC2', 'BCAS2', 'BRD8', 'NOC2L', 'ACVR1', 'MED7', 'ODR4', 'ILRUN', 'PSMA1', 'REEP6', 'GDPD5', 'NONO', 'UBXN1', 'LPXN', 'CLEC11A', 'KRBA1', 'C4orf46', 'RIC8A', 'PHC2', 'DLG1', 'XPO6', 'MGAT2', 'ATP5ME', 'VPS13D', 'TOM1', 'HIKESHI', 'ETHE1', 'SGTB', 'PDPR', 'ALDH1L1', 'ZNF689', 'UTP6', 'CEP63', 'GNB1L', 'MYO6', 'RNPS1', 'IMP4', 'APPBP2', 'GBF1', 'CORO1B', 'BAIAP2L1', 'PPIP5K2', 'CRTC3', 'COA7', 'KIF18A', 'PRPSAP2', 'RSL1D1', 'CDH24', 'OPA1', 'KLHL12', '